\textbf{LABORATORIUM NR 3 - REGRESJA LOGISTYCZNA}

Na dzisiejszych zajęciach poznamy regresję logistyczną, która jest jednym z najprostszych klasyfikatorów. Dowiemy się również, czym są one-hot vectory, a także poznamy technikę kross-walidacji (ang. cross-validation).

Zacznijmy od pobrania odpowiednich bibliotek.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

Wczytajmy plik z danymi. Dane te zawierają informacje o rekrutacji studentów na studia II stopnia. Pierwsza z kolumn to informacja, czy student dostał się na studia (1), czy też nie (0). Kolumna "gre" przedstawia liczbę punktów zdobytych w końcowym teście, kolumna "gpa" dotyczy średniej ocen, zaś kolumna "rank" odpowiada randze szkoły, w której student uczył się wcześniej. Przyjmuje wartości ze zbioru liczb: {1, 2, 3, 4}, przy czym im mniejsza wartość, tym lepsza szkoła. Parametr \textit{skiprows} z wartością 1 dotyczy ominięcia pierwszego wiersza, który ma nagłówki kolumn.

In [4]:
data = np.loadtxt('logistyczna.csv', delimiter=',', skiprows=1)

Wyrazimy ostatnią kolumnę w postaci one-hot wektorów, czyli wektorów wypełnionych zerami o rozmiarze takim jak liczba dostępnych klas, mającymi jedynkę w miejscu, które odpowiada numerkowi klasy, tzn.: klasie 1 odpowiada wektor (1, 0, 0, 0), klasie 2 (0, 1, 0, 0), klasie 3 (0, 0, 1, 0) zaś klasie 4 odpowiada wektor (0, 0, 0, 1). 

In [5]:
note = data[:, 3].astype(int)
note = note - 1  # Przesuwamy zakres z {1, ..., 4} do {0, ..., 3}
print(note)

[2 2 0 3 3 1 0 1 2 1 3 0 0 1 0 2 3 2 1 0 2 1 3 3 1 0 0 3 1 0 3 2 2 2 0 1 0
 2 1 2 1 1 1 2 1 2 1 3 3 2 2 3 3 1 2 2 2 2 1 3 1 3 2 2 2 1 3 0 0 0 2 3 3 1
 3 2 2 2 0 0 3 1 1 3 2 1 1 1 0 1 1 0 1 1 1 1 3 1 1 2 2 2 3 2 1 1 0 1 2 1 3
 3 2 0 2 2 1 1 0 2 1 1 2 2 2 3 0 3 1 3 1 1 1 2 1 2 3 2 1 0 1 3 3 2 3 2 1 2
 0 0 0 1 1 2 2 3 1 0 1 2 1 1 1 1 1 0 3 2 2 2 2 2 2 1 3 1 1 2 2 2 2 3 1 1 3
 1 2 1 1 1 1 2 2 3 1 1 2 3 2 3 2 1 0 3 0 2 0 0 2 1 3 1 1 2 1 2 0 0 0 1 2 2
 0 2 1 2 1 3 1 1 3 2 1 2 0 1 1 1 3 2 1 0 2 1 0 2 1 1 2 2 3 3 1 3 3 2 1 2 1
 1 1 1 2 2 2 2 3 2 1 2 1 2 1 0 1 1 2 0 3 1 1 2 3 3 1 3 0 3 3 3 1 1 1 0 0 2
 0 1 1 2 1 2 1 1 2 3 0 1 1 2 2 1 2 3 3 1 1 3 3 0 2 1 3 1 2 0 1 1 1 3 2 2 0
 2 2 0 2 3 0 2 3 2 3 1 2 2 1 1 1 1 1 2 2 1 1 0 1 0 2 2 0 0 1 1 0 2 2 2 0 1
 1 2 0 0 1 3 1 1 2 1 1 1 1 0 1 0 1 1 1 1 1 1 2 1 2 1 2 1 1 2]


In [6]:
one_hot_note = np.eye(4)[note]
print(one_hot_note)

[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


Usuńmy teraz oryginalną kolumnę odpowiadającą wartościom "rank" i wstawmy tam przygotowane one-hot wektory.

In [7]:
data = np.delete(data, 3, axis=1)
data = np.concatenate((data, one_hot_note), axis=1)
print(data)

[[  0.   380.     3.61 ...   0.     1.     0.  ]
 [  1.   660.     3.67 ...   0.     1.     0.  ]
 [  1.   800.     4.   ...   0.     0.     0.  ]
 ...
 [  0.   460.     2.63 ...   1.     0.     0.  ]
 [  0.   700.     3.65 ...   1.     0.     0.  ]
 [  0.   600.     3.89 ...   0.     1.     0.  ]]


Rozdzielmy teraz dane na argumenty i wartości funkcji.

In [8]:
X = data[:, 1:]
y = data[:, 0]

Podzielmy dane na zbiór treningowy i testowy.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=2)

Przygotujmy funkcję do treningu regresji logistycznej.

In [10]:
def prepare_logistic_regression(seed,
                                epochs,
                                X_train,
                                y_train,
                                X_test,
                                y_test):
    model = LogisticRegression(random_state=seed,
                               solver='lbfgs',
                               max_iter=epochs,
                               multi_class='multinomial').fit(X_train,
                                                              y_train)
    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)
    print(f'''Dokladnosc klasyfikacji dla zbioru treningowego wynosi {train_accuracy},
              zas dokladnosc klasyfikacji dla zbioru testowego wynosi {test_accuracy}.''')
    return model

In [11]:
model = prepare_logistic_regression(1, 500, X_train, y_train, X_test, y_test)

Dokladnosc klasyfikacji dla zbioru treningowego wynosi 0.725,
              zas dokladnosc klasyfikacji dla zbioru testowego wynosi 0.7.


Dokonajmy teraz standaryzacji danych, czyli odejmijmy średnią i podzielmy przez odchylenie standardowe.
\textbf{UWAGA! Częsty błąd!} Preprocessing danych wykonujemy na danych treningowych i przygotowane w ten sposób parametry aplikujemy do zbioru testowego! Nie wykonujemy standaryzacji na całym zbiorze danych jednocześnie! 

In [12]:
standarization = preprocessing.StandardScaler().fit(X_train)
X_train_standarized = standarization.transform(X_train)
X_test_standarized = standarization.transform(X_test)

In [13]:
print(f'Przykładowe dane przed standaryzacją: \n {X_train[:10]}')
print(f'Przykładowe dane po standaryzacji: \n {X_train_standarized[:10]}')

Przykładowe dane przed standaryzacją: 
 [[360.     3.     0.     0.     1.     0.  ]
 [580.     3.5    0.     1.     0.     0.  ]
 [480.     2.55   1.     0.     0.     0.  ]
 [580.     3.51   0.     1.     0.     0.  ]
 [500.     3.03   0.     0.     1.     0.  ]
 [660.     4.     0.     1.     0.     0.  ]
 [580.     3.77   0.     0.     0.     1.  ]
 [540.     3.49   1.     0.     0.     0.  ]
 [600.     3.64   0.     0.     1.     0.  ]
 [600.     3.62   0.     0.     1.     0.  ]]
Przykładowe dane po standaryzacji: 
 [[-2.04086945 -0.97934481 -0.40824829 -0.78050971  1.50193673 -0.45485883]
 [-0.08020884  0.31559544 -0.40824829  1.28121405 -0.66580701 -0.45485883]
 [-0.97141821 -2.14479104  2.44948974 -0.78050971 -0.66580701 -0.45485883]
 [-0.08020884  0.34149424 -0.40824829  1.28121405 -0.66580701 -0.45485883]
 [-0.79317634 -0.9016484  -0.40824829 -0.78050971  1.50193673 -0.45485883]
 [ 0.63275865  1.61053569 -0.40824829  1.28121405 -0.66580701 -0.45485883]
 [-0.08020884  1.01486

In [14]:
model = prepare_logistic_regression(1, 500, X_train_standarized, y_train,
                                    X_test_standarized, y_test)

Dokladnosc klasyfikacji dla zbioru treningowego wynosi 0.7214285714285714,
              zas dokladnosc klasyfikacji dla zbioru testowego wynosi 0.7166666666666667.


Często stosowaną techniką w uczeniu maszynowym jest \textbf{k-krotny sprawdzian krzyżowy} (ang. k-fold cross-validation). Zwykle stosujemy ją, gdy mamy niedobór danych, a chcemy uzyskać reprezentatywny wynik klasyfikatora. Procedura ta polega na tym, że dzielmy dane na \textit{k} podzbiorów i trenujemy nasz klasyfikator \textit{k} razy, za każdym razem biorąc wybrany podzbiór jako zbiór testowy, a pozostałe \textit{k - 1} podzbiorów jako zbiór treningowy. Dla każdego $i \in \lbrace 1, 2, ..., k\rbrace$ inny podzbiór znajduje się w zbiorze testowym, a zbiór treningowy stanowi połączenie pozostałych \textit{k - 1} podzbiorów.   

In [15]:
from sklearn.model_selection import cross_val_score
classifier = LogisticRegression(random_state=5,
                                solver='lbfgs',
                                max_iter=500,
                                multi_class='multinomial')
scores = cross_val_score(classifier, X, y, cv=5)
print(scores)

[0.7125 0.725  0.7    0.675  0.7   ]


In [16]:
print(f'Średnia dokładność klasyfikacji wynosi: {scores.mean():.3f}, zaś odchylenie standardowe {scores.std():.3f}')

Średnia dokładność klasyfikacji wynosi: 0.703, zaś odchylenie standardowe 0.017
